<a href="https://colab.research.google.com/github/muthuraman2002/vector_Db_Learning/blob/main/chatbot_using_chormaDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
with open('your_document.txt', 'w') as f:
  f.write('This is some text for the file.\n')
  f.write('Another line of text.')

In [1]:
# @title Default title text
!pip install langchain chromadb tiktoken langchain-community transformers tensorflow sentence-transformers



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opentelemetry-proto to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 40.2 MB/s eta 0:00:00
   ━━━━━

In [1]:
import os
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import HuggingFacePipeline
from langchain.chains import ConversationalRetrievalChain
# from transformers import pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from sentence_transformers import SentenceTransformer


In [10]:
# print(embeddings,'try')
# Load your documents (replace with your actual document loading)
file_path = "my_document.txt"  # Replace with your file path
text=""
try:
    with open(file_path, "r") as f:
      # print(f.read(),'reading')
      text = f.read()
except FileNotFoundError:
    with open(file_path, "w") as f:
      f.write("")  # Optionally initialize with an empty string
      text = ""

# texts = [
#     "The quick brown fox jumps over the lazy dog.",
#     "A journey of a thousand miles begins with a single step.",
#     "To be or not to be, that is the question.",
#     "All that glitters is not gold."
# ]
# print(pipeline)
# Split the text into chunks

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(text)
# Create embeddings and store them in Chroma
# model= SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")  # Use HuggingFaceEmbeddings
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
# embeddings = model.encode(texts)
# print(texts,'texts')
metadatas = [{"source": str(i)} for i in range(len(texts))]
# print(texts,Chroma.from_texts())
# try:
# docsearch = Chroma.afrom_texts(texts, embeddings, metadatas=metadatas)
docsearch = Chroma.from_texts(
    texts=texts,
    embedding=embeddings,
    metadatas=metadatas,
    persist_directory="./chroma_data"  # Optional for persistence
)
# except Exception as e:
#     print(f"An error occurred: {e}",'exception')
model_id = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=100)
llm = HuggingFacePipeline(pipeline=pipe)
# Initialize the language model using Hugging Face's pipeline
# hf_pipeline = pipeline("text-generation", model="gpt2")
# print(HuggingFacePipeline,'tokeniser')
# llm = HuggingFacePipeline(pipeline=hf_pipeline)
# pad_token_id=hf_pipeline.tokenizer.eos_token_id
# print(pad_token_id)
# Create the conversational retrieval chain
# try:
qa = ConversationalRetrievalChain.from_llm(llm, docsearch.as_retriever())
# except Exception as e:
    # print(f'error occured :{e}','qa')
    # Start the chat loop
# print(qa)
try:
    chat_history = []
    while True:
        query = input("Ask a question (or type 'exit' to quit): ")
        if query.lower() == "exit":
            break

        result = qa({"question": query, "chat_history": chat_history})
        print(result["answer"])
        chat_history.append((query, result["answer"]))
    print(f"Chat History: {chat_history}")
except Exception as e:
    print(f"An error occurred: {e} question")
    # print('hii')


Ask a question (or type 'exit' to quit): what is model ?


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (1024). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


An error occurred: index out of range in self question
